# 解壓縮

In [ ]:
!rm -rf dataset_xy
!rm -rf output
!unzip -q road_following_dataset_xy_2024-11-03_12-08-48.zip

# 函式庫

In [ ]:
!pip uninstall scipy




In [4]:
!pip install scipy

Looking in indexes: https://mirror.sjtu.edu.cn/pypi/web/simple



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\Chino\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip3 --version
!pip3 install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu121

pip 21.3.1 from C:\Users\Chino\Documents\Methylation_analysis_prostate_cancer\venv\lib\site-packages\pip (python 3.10)

Looking in indexes: https://download.pytorch.org/whl/cu121


You should consider upgrading via the 'C:\Users\Chino\Documents\Methylation_analysis_prostate_cancer\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn.functional as F
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn as nn


# 檔案格式解碼


In [2]:
def get_x(path, width):
    """Gets the x value from the image filename"""
    return (float(int(path.split("_")[1])) - width/2) / (width/2)

def get_y(path, height):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[2])) - height/2) / (height/2)

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        width, height = image.size
        x = float(get_x(os.path.basename(image_path), width))
        y = float(get_y(os.path.basename(image_path), height))
      
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('1100', random_hflips=False)
print(dataset[0][1])

tensor([0.9018, 0.7054])


C:\Users\Chino\AppData\Local\Temp\ipykernel_61936\3035275046.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:


# 區分訓練集


In [7]:
test_percent = 0.2
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

# 提早結束


In [8]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


# 訓練


In [9]:
import torch
print(torch.__version__)
print(torch.version.cuda)  # 查看 PyTorch 所使用的 CUDA 版本
print(torch.cuda.is_available())  # 檢查是否可使用 CUDA


2.5.0+cu121
12.1
True


In [22]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import scipy
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)
NUM_EPOCHS = 40
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

C:\Users\Chino\AppData\Local\Temp\ipykernel_15512\3035275046.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:


0.183674, 0.027080
0.037527, 0.011598
0.025441, 0.027195
0.022191, 0.016323
0.010271, 0.010801
0.010094, 0.009909
0.007082, 0.006737
0.010999, 0.010518
0.009786, 0.006716
0.007229, 0.007962
0.007048, 0.006787
0.008507, 0.006765
0.006415, 0.007098
0.005395, 0.007559
0.006857, 0.006943
0.004813, 0.007227
0.006151, 0.007539
0.004676, 0.006225
0.005738, 0.005222
0.004904, 0.007758
0.004526, 0.004814
0.005273, 0.007263
0.004427, 0.005209
0.004980, 0.006562
0.005174, 0.008161
0.005986, 0.006507
0.005013, 0.008764
0.003806, 0.006984
0.003806, 0.006883
0.003666, 0.006932
0.004157, 0.009073
0.006385, 0.006268
0.004135, 0.005613
0.003219, 0.005438
0.002588, 0.004988
0.003066, 0.005204
0.015437, 0.104340
0.023970, 0.011067
0.008832, 0.007699
0.006551, 0.010238


In [26]:
import time
import torch
import torchvision.transforms as transforms
import cv2
import os
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# 設定資料夾路徑
image_folder = "1100"
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# 設定 widget 以顯示圖片
widget_width = 224
widget_height = 224
image_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)
display(image_widget)

# 設定裝置 (若有 GPU 可用)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 加載模型
model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("lane_detection_model.pth", map_location=device))
model = model.to(device)
model.eval()  # 將模型設定為推論模式

# 圖片預處理和顯示函數
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4691, 0.4032, 0.4579], [0.1740, 0.1485, 0.1688])
])

def bgr8_to_jpeg(image):
    _, jpeg = cv2.imencode('.jpg', image)
    return jpeg.tobytes()

def process_image(image_path):
    # 讀取圖片並進行預處理
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(input_tensor)
        x, y = output[0].cpu().numpy()
    print(x,y)
    # 映射預測的 (x, y) 到圖片的像素坐標，假設 x 和 y 是 [0, 1] 範圍內的預測
    x_pixel = int(x * widget_width / 2 + widget_width / 2)
    y_pixel = int(y * widget_height / 2 + widget_height / 2)
    
    # 在圖片上繪製預測結果
    display_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # 確保格式正確
    cv2.circle(display_image, (x_pixel, y_pixel), 5, (0, 255, 0), -1)  # 綠色點表示預測位置
    
    # 更新 widget 顯示處理後的圖片
    image_widget.value = bgr8_to_jpeg(display_image)

# 對資料夾中的每張圖片進行處理
for image_file in image_files:
    process_image(image_file)
    time.sleep(1)  # 暫停以觀察每張圖片的結果


Image(value=b'', format='jpeg', height='224', width='224')

cuda


C:\Users\Chino\AppData\Local\Temp\ipykernel_15512\1303108273.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lane_detection_model.pth"

-0.8780172 0.717673
-0.8778583 0.7950877
-0.9178627 0.84937817
-0.85928243 0.5425702
-0.88263667 0.59387165
-0.880228 0.51428795
-0.8686982 0.5173032
-0.8280813 0.6212539
-0.8869151 0.7009606
-0.87433624 0.73298657
-0.90913576 0.73528033
-0.8728177 0.7922363
-0.8945904 0.79394495
-0.8679162 0.59366095
-0.8702812 0.6430227
-0.89169306 0.68821
-0.8203165 0.72577375
-0.8397779 0.77373385
-0.8263668 0.7792012
-0.8380804 0.6266011
-0.8518399 0.68623847
-0.87821007 0.8074782
-0.83594745 0.6760291
-0.90653384 0.72998506
-0.8525966 0.7429267
-0.8472894 0.62026083
-0.7981664 0.59726024
-0.8309813 0.6323238
-0.835523 0.65371215
-0.832945 0.78938013
-0.83614784 0.61466235
-0.75203913 0.6402804
-0.80812865 0.63145787
-0.7673331 0.65914136
-0.7881774 0.6809462
-0.85062516 0.713171
-0.79798067 0.67826164
-0.8031116 0.70067656
-0.84401244 0.74708265
-0.849312 0.615506
-0.8171975 0.6445993
-0.82427174 0.6531872
-0.842652 0.63734204
-0.842652 0.63734204
-0.92720217 0.72290987
-0.8408798 0.6862918
-0.78

KeyboardInterrupt: 

In [1]:
import torch
import torch.nn as nn
from torchvision import models
import torch.nn.utils.prune as prune

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 初始化 ResNet-18 模型并调整最后一层以匹配您的任务（假设输出为2个类别）
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, 2)  # 根据您的任务调整输出类别数
model = model.to(device)

# 加载已训练的权重
model_path = "lane_detection_model.pth"  # 替换为您的模型文件路径
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()  # 设置模型为推理模式

# 定义剪枝函数
def prune_model(model, amount=0.2):
    """
    对模型的卷积层和全连接层应用剪枝。
    :param model: 要剪枝的模型
    :param amount: 剪枝比例，例如0.2表示剪掉20%的参数
    """
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')  # 移除剪枝的重参数，保留剪枝后的权重

# 应用剪枝
prune_model(model, amount=0.2)

# 测试剪枝后的模型
with torch.no_grad():
    test_input = torch.randn(1, 3, 224, 224).to(device)  # 根据您的输入尺寸调整
    output = model(test_input)
    print("Model output after pruning:", output)

# 保存剪枝后的模型
torch.save(model.state_dict(), "pruned_lane_detection_model.pth")
print("Pruned model saved as pruned_lane_detection_model.pth")


Using device: cuda


C:\Users\Chino\AppData\Local\Temp\ipykernel_63304\326031744.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=de

Model output after pruning: tensor([[-0.2314,  0.6814]], device='cuda:0')
Pruned model saved as pruned_lane_detection_model.pth
